In [33]:
# PASO 1: CONFIGURACIÓN INICIAL
!pip install pandas numpy scipy openpyxl
import pandas as pd
import numpy as np
from scipy import stats
# -----------------------------------------------------------
# PASO 2: CARGA Y PREPARACIÓN DE DATOS
# Reemplazar con tu ruta exacta
ruta = '/content/data.csv'
df = pd.read_csv(ruta, sep=',', decimal='.',encoding='latin-1')



In [35]:
# Paso 4: Filtrar por Sectores Seleccionados
sectores_filtrados = ['INDUSTRIAS MANUFACTURERAS','COMERCIO AL POR MAYOR Y AL POR MENOR']
df = df[df['ciiu_agregado'].isin(sectores_filtrados)]

In [36]:

variables_monetarias = ['ventas_netas', 'valor_mp', 'valor_activos']
df[variables_monetarias] = df[variables_monetarias].replace(r'^\s*$', np.nan, regex=True)
df[variables_monetarias] = df[variables_monetarias].apply(pd.to_numeric, errors='coerce')


# Paso 5: Transformar Variables Monetarias
#variables_monetarias = ['ventas_netas', 'valor_mp', 'valor_remuneracion', 'valor_terceros']
for var in variables_monetarias:
    df[f"log_{var}"] = np.log(df[var] + 1)  # Aplicar log(x+1)

# Paso 6: Función para Calcular Diferencias de Medias
def calcular_estadisticas(data, variable, grupo):
    grupo0 = data[data[grupo] == 0][variable].dropna()
    grupo1 = data[data[grupo] == 1][variable].dropna()

# Cálculo de medias y test t para diferencias
    ttest = stats.ttest_ind(grupo0, grupo1, equal_var=False)

    return {
        'Variable': variable,
        'Media s/certificación': np.mean(grupo0),
        'Media c/certificación': np.mean(grupo1),
        'Diferencia': np.mean(grupo1) - np.mean(grupo0),
        'p-valor': ttest.pvalue,
        'N° Firmas': len(grupo0) + len(grupo1)  # Nueva columna
    }

# Paso 7: Definir Variables del Diccionario
variables_diccionario = [
    'L_promedio', 'capacidad_instalada', 'log_ventas_netas', 'log_valor_mp',
    'D_capacitacion', 'D_inversion_maq_eq', 'D_exportacion', 'D_credito',
    'D_tec_1', 'D_egrande', 'D_emediano', 'D_epequeño', 'D_internet', 'D_trabajo_1', 'D_trabajo_2',
    'D_calidad', 'D_mercado', 'D_exclusivo', 'D_orden_compra', 'D_pagina_web', 'D_exp_gerente'
]

# Paso 8: Calcular Resultados para Todas las Variables
resultados = []
for variable in variables_diccionario:
    try:
        resultado = calcular_estadisticas(df, variable, 'D_certificacion')
        resultados.append(resultado)
    except Exception as e:
        print(f"Error en la variable {variable}: {e}")

# Paso 9: Convertir Resultados en DataFrame
resultados_df = pd.DataFrame(resultados)

# Paso 10: Agregar Significancia al Resultado de Diferencia
def aplicar_significancia(row):
    if row['p-valor'] < 0.01:
        return f"{row['Diferencia']:.2f}***"
    elif row['p-valor'] < 0.05:
        return f"{row['Diferencia']:.2f}**"
    elif row['p-valor'] < 0.1:
        return f"{row['Diferencia']:.2f}*"
    else:
        return f"{row['Diferencia']:.2f}"

resultados_df['Diferencia'] = resultados_df.apply(aplicar_significancia, axis=1)
#resultados_df = resultados_df.drop(columns=['p-valor'])  # Eliminar columna de p-valor si no es necesaria


Error en la variable D_epequeño: 'D_epequeño'


/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [37]:
# Paso 11: Exportar Resultados a Excel
ruta_salida = '/content/Diferencias_Medias.xlsx'
resultados_df.to_excel(ruta_salida, index=False)
print(f"Resultados exportados a: {ruta_salida}")


Resultados exportados a: /content/Diferencias_Medias.xlsx
